In [ ]:
!conda install -c conda-forge gdcm -y

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import pydicom
import scipy.ndimage
import gdcm

from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage.morphology import disk, opening, closing
from tqdm import tqdm

from IPython.display import HTML
from PIL import Image

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from os import listdir, mkdir

## Table of contents

1. [Qu'est-ce que la fibrose pulmonaire ?](#fibrosis)
2. [References](#references)
    * [Data Science Bowl 2017 - Preprocessing Tutorial by Guido Zuidhof](#bowl_2017)
    * [Papers](#papers)
2. [Data paths](#prepare)
3. [Travailler avec des fichiers dicom](#dicom)
    * [Chargement des CT-scans par patient](#ct_scans)
    * [Transformation en unités Hounsfield](#hunits)
    * [La taille des voxels](#voxel)
    * [Surface et volume de la tranche du CT-scan - EDA](#scan_eda)
    * [Reconstruction 3D des scanners](#reconstruction)
    * [Segmentation des tissus](#segmentation)
4. [Génération d'un ensemble de données pour les fichiers prétraités](#datagenerator)
    * [Lien vers le data-set](https://www.kaggle.com/allunia/osic-pulmonary-fibrosis-progression-huscans)

# Qu'est-ce que la fibrose pulmonaire ?

![](https://hopital-prive-saint-martin-caen.ramsaygds.fr/sites/default/files/styles/article_header_desktop/public/pneumo_fibrose_pulmonaire.jpg?itok=WNqdaZFm)


#### **Quand parle-t-on de fibrose pulmonaire ?**

    On parle de fibrose pulmonaire lorsque se développe dans le poumon du tissu fibreux qui remplace peu à peu le tissu normal. Certaines fibroses pulmonaires résultent d’une toxicité de médicaments ; d’autres sont associées à des maladies auto-immunes, c’est-à-dire des maladies dans lesquelles notre système immunitaire attaque nos organes.
    Parfois, on ne retrouve pas de cause et on parle de « fibrose pulmonaire idiopathique » (FPI). La FPI est une maladie rare (1 personne sur 2 500 à 1 sur 7 000 personnes), d’origine inexpliquée, qu’on rencontre plus volontiers après la soixantaine.

#### **Comment se manifeste la fibrose pulmonaire ?**

    La fibrose pulmonaire se traduit par un essoufflement progressif et une toux sèche. Les patients peuvent aussi présenter un amaigrissement, une perte d’appétit, une fatigue importante. Dans un cas sur deux, les doigts revêtent un aspect caractéristique en baguette de tambour, avec des ongles bombés.


#### **Comment diagnostique-t-on la fibrose pulmonaire ?**

    Le diagnostic de fibrose pulmonaire est souvent difficile. À l’auscultation, le médecin peut entendre des bruits pulmonaires évocateurs.

    La radiographie pulmonaire peut être normale au début. Mais, le scanner visualise, dans un cas sur deux, les zones de fibrose sous forme d’un aspect « en rayon de miel ».

    Pour écarter d’autres maladies (maladie liée à l’amiante, silicose des mineurs), on pourra effectuer un lavage des bronches et des alvéoles, ce qui nécessite une fibroscopie ; ce lavage permet de recueillir des cellules pulmonaires pour les analyser.

    Parfois, une biopsie de poumon sera demandée. Cet examen exige un geste chirurgical.
    Afin d’évaluer le degré de handicap respiratoire, on a recours à un test de marche (capacité à l’effort), des épreuves fonctionnelles respiratoires (mesure du souffle), une mesure des gaz sanguins (oxygène, gaz carbonique). 


# References <a class="anchor" id="references"></a>



* [Intrinsic dependencies of CT radiomic features on voxel size and number of gray levels](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5462462/)

# Data paths <a class="anchor" id="prepare"></a>

In [ ]:
listdir("../input/")

In [ ]:
#basepath = "../input/osic-pulmonary-fibrosis-progression/"
# or if you are taking part in RSNA pulmonary embolism detection:
basepath = "../input/rsna-str-pulmonary-embolism-detection/"
listdir(basepath)

Let's load the csv-files:

In [ ]:
train = pd.read_csv(basepath + "train.csv")
test = pd.read_csv(basepath + "test.csv")

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
if basepath == "../input/osic-pulmonary-fibrosis-progression/":
    train["dcm_path"] = basepath + "train/" + train.Patient + "/"
else:
    train["dcm_path"] = basepath + "train/" + train.StudyInstanceUID + "/" + train.SeriesInstanceUID  

# Travailler avec des fichiers dicom <a class="anchor" id="dicom"></a>

## Chargement des CT-scans par patient <a class="anchor" id="ct_scans"></a>

* Pour charger le scan 3D complet, nous devons commander les fichiers/tranches dicom uniques par "ImagePositionPatient": 

In [ ]:
def load_scans(dcm_path):
    if basepath == "../input/osic-pulmonary-fibrosis-progression/":
        # in this competition we have missing values in ImagePosition, this is why we are sorting by filename number
        files = listdir(dcm_path)
        file_nums = [np.int(file.split(".")[0]) for file in files]
        sorted_file_nums = np.sort(file_nums)[::-1]
        slices = [pydicom.dcmread(dcm_path + "/" + str(file_num) + ".dcm" ) for file_num in sorted_file_nums]
    else:
        # otherwise we sort by ImagePositionPatient (z-coordinate) or by SliceLocation
        slices = [pydicom.dcmread(dcm_path + "/" + file) for file in listdir(dcm_path)]
        slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    return slices

In [ ]:
example = train.dcm_path.values[0]
scans = load_scans(example)

Examinons le premier fichier dicom de notre exemple de patient :

In [ ]:
scans[0]

### information

1. Le CT-scan capture des informations sur la radiodensité d'un objet ou d'un tissu exposé aux rayons X. Une tranche transversale d'un scan est reconstituée après avoir pris des mesures dans plusieurs directions différentes.
2. Nous devons passer à des unités Hounsfield car la composition spectrale des rayons X dépend des paramètres de mesure tels que les paramètres d'acquisition et la tension du tube. En se normalisant aux valeurs de l'eau et de l'air (l'eau a un HU 0 et l'air -1000), les images des différentes mesures deviennent comparables.
3. Un ct-scanner donne environ 4000 valeurs de gris qui ne peuvent pas être capturées par nos yeux. C'est pourquoi on procède à un fenêtrage. De cette façon, l'image est affichée dans une plage de HU qui correspond le mieux à la région d'intérêt. 

## Transformation en unités Hounsfield <a class="anchor" id="hunits"></a>

Avant de commencer, traçons la distribution des pixels de certains fichiers dicom pour avoir une impression des données brutes:

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
for n in range(10):
    image = scans[n].pixel_array.flatten()
    rescaled_image = image * scans[n].RescaleSlope + scans[n].RescaleIntercept
    sns.distplot(image.flatten(), ax=ax[0]);
    sns.distplot(rescaled_image.flatten(), ax=ax[1])
ax[0].set_title("Raw pixel array distributions for 10 examples")
ax[1].set_title("HU unit distributions for 10 examples");

Pour quelques exemples, nous pouvons voir qu'il existe des valeurs brutes à -2000. Elles correspondent à des images ayant une limite circulaire à l'intérieur de l'image. L'"extérieur" de cette valeur circulaire est souvent fixé par défaut à -2000 (ou dans d'autres concours, j'ai également trouvé -3000).

In [ ]:
def transform_to_hu(slices):
    images = np.stack([file.pixel_array for file in slices])
    images = images.astype(np.int16)

    # convert ouside pixel-values to air:
    # I'm using <= -1000 to be sure that other defaults are captured as well
    images[images <= -1000] = 0
    
    # convert to HU
    for n in range(len(slices)):
        
        intercept = slices[n].RescaleIntercept
        slope = slices[n].RescaleSlope
        
        if slope != 1:
            images[n] = slope * images[n].astype(np.float64)
            images[n] = images[n].astype(np.int16)
            
        images[n] += np.int16(intercept)
    
    return np.array(images, dtype=np.int16)

In [ ]:
hu_scans = transform_to_hu(scans)

In [ ]:
fig, ax = plt.subplots(1,4,figsize=(20,3))
ax[0].set_title("Original CT-scan")
ax[0].imshow(scans[0].pixel_array, cmap="bone")
ax[1].set_title("Pixelarray distribution");
sns.distplot(scans[0].pixel_array.flatten(), ax=ax[1]);

ax[2].set_title("CT-scan in HU")
ax[2].imshow(hu_scans[0], cmap="bone")
ax[3].set_title("HU values distribution");
sns.distplot(hu_scans[0].flatten(), ax=ax[3]);

for m in [0,2]:
    ax[m].grid(False)

### Tri de nos tranches et création de GIF

In [ ]:
first_patient = load_slice('../input/rsna-str-pulmonary-embolism-detection/train/0003b3d648eb/d2b2960c2bbf')
first_patient_pixels = transform_to_hu(first_patient)

def sample_stack(stack, rows=6, cols=6, start_with=10, show_every=5):
    fig,ax = plt.subplots(rows,cols,figsize=[18,20])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title(f'slice {ind}')
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='bone')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

sample_stack(first_patient_pixels)

Maintenant, toutes les valeurs brutes par tranche sont mises à l'échelle des H-units.

## La taille des voxels <a class="anchor" id="voxel"></a>

Le voxel représente le pixel 3D qui est donné dans un scanner. Pour autant que je sache, il est couvert par le plan 2D de l'attribut d'espacement des pixels dans les directions x et y et par l'épaisseur de la tranche dans la direction z.

### Pixelspacing

* L'attribut "pixelspacing" que vous pouvez trouver dans les fichiers dicom est important. Il nous indique la distance physique parcourue par un pixel. Vous pouvez voir qu'il n'y a que 2 valeurs qui décrivent les directions x et y dans le plan d'une tranche transversale. 
* Pour un patient, cet espacement des pixels est généralement le même pour toutes les tranches.
* Mais entre les patients, l'espacement des pixels peut varier en raison des préférences personnelles ou institutionnelles des médecins et de la clinique, et il dépend également du type de scanner. Par conséquent, si vous comparez deux images dans la taille des poumons, cela ne signifie pas automatiquement que la plus grande est vraiment plus grande dans la taille physique de l'organe !

Examinons les distributions des largeurs et des hauteurs d'espacement des pixels des patients

In [ ]:
N = 100

Pour accélérer le calcul, j'ai sélectionné N patients à prendre en considération. Utilisez N = train.shape [0] pour le faire pour tous les patients de l'ensemble de données

In [ ]:
def get_window_value(feature):
    if type(feature) == pydicom.multival.MultiValue:
        return np.int(feature[0])
    else:
        return np.int(feature)

pixelspacing_r = []
pixelspacing_c = []
slice_thicknesses = []
patient_id = []
patient_pth = []
row_values = []
column_values = []
window_widths = []
window_levels = []

if basepath == "../input/osic-pulmonary-fibrosis-progression/":
    patients = train.Patient.unique()[0:N]
else:
    patients = train.SeriesInstanceUID.unique()[0:N]

for patient in patients:
    patient_id.append(patient)
    if basepath == "../input/osic-pulmonary-fibrosis-progression/":
        path = train[train.Patient == patient].dcm_path.values[0]
    else:
        path = train[train.SeriesInstanceUID == patient].dcm_path.values[0]
    example_dcm = listdir(path)[0]
    patient_pth.append(path)
    dataset = pydicom.dcmread(path + "/" + example_dcm)
    
    window_widths.append(get_window_value(dataset.WindowWidth))
    window_levels.append(get_window_value(dataset.WindowCenter))
    
    spacing = dataset.PixelSpacing
    slice_thicknesses.append(dataset.SliceThickness)
    
    row_values.append(dataset.Rows)
    column_values.append(dataset.Columns)
    pixelspacing_r.append(spacing[0])
    pixelspacing_c.append(spacing[1])
    
scan_properties = pd.DataFrame(data=patient_id, columns=["patient"])
scan_properties.loc[:, "rows"] = row_values
scan_properties.loc[:, "columns"] = column_values
scan_properties.loc[:, "area"] = scan_properties["rows"] * scan_properties["columns"]
scan_properties.loc[:, "pixelspacing_r"] = pixelspacing_r
scan_properties.loc[:, "pixelspacing_c"] = pixelspacing_c
scan_properties.loc[:, "pixelspacing_area"] = scan_properties.pixelspacing_r * scan_properties.pixelspacing_c
scan_properties.loc[:, "slice_thickness"] = slice_thicknesses
scan_properties.loc[:, "patient_pth"] = patient_pth
scan_properties.loc[:, "window_width"] = window_widths
scan_properties.loc[:, "window_level"] = window_levels
scan_properties.head()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.distplot(pixelspacing_r, ax=ax[0], color="Limegreen", kde=False)
ax[0].set_title("Pixel spacing distribution \n in row direction ")
ax[0].set_ylabel("Counts in train")
ax[0].set_xlabel("mm")
sns.distplot(pixelspacing_c, ax=ax[1], color="Mediumseagreen", kde=False)
ax[1].set_title("Pixel spacing distribution \n in column direction");
ax[1].set_ylabel("Counts in train");
ax[1].set_xlabel("mm");

On voit que les valeurs varient vraiment beaucoup d'un patient à l'autre ! Comme elles sont données en mm et que les scans ct couvrent généralement 512 valeurs de lignes et de colonnes... **Nous pouvons calculer la distance minimale et maximale couverte par les images

### Épaisseur de la tranche et surface des pixels

L'épaisseur de la tranche nous indique la distance parcourue par une tranche dans la direction Z. Traçons également la distribution de celle-ci. En outre, le tableau de pixels des valeurs brutes couvre une zone spécifique donnée par les valeurs des lignes et des colonnes. Examinons-le également

In [ ]:
counts = scan_properties.groupby(["rows", "columns"]).size()
counts = counts.unstack()
counts.fillna(0, inplace=True)


fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.distplot(slice_thicknesses, color="orangered", kde=False, ax=ax[0])
ax[0].set_title("Slice thicknesses of all patients");
ax[0].set_xlabel("Slice thickness in mm")
ax[0].set_ylabel("Counts in train");

for n in counts.index.values:
    for m in counts.columns.values:
        ax[1].scatter(n, m, s=counts.loc[n,m], c="midnightblue")
ax[1].set_xlabel("rows")
ax[1].set_ylabel("columns")
ax[1].set_title("Pixel area of ct-scan per patient");

* Des tranches très fines permettent de montrer plus de détails. En revanche, les tranches épaisses contiennent moins de bruit mais sont plus sujettes aux artefacts. Hmm... Je suis très excité de voir quelques exemples ici aussi. 
* Même s'il est courant d'avoir des zones de taille 512x512 pixels, on peut voir que ce n'est pas toujours vrai ! On peut trouver beaucoup d'exceptions et même une ou quelques très grandes zones de pixels (1300x1300) ! !! (OSIC)
* Un prétraitement correct de ces scans pourrait être très important... nous devons le vérifier.

## Zone physique et volume de la tranche couverts par un seul ct-scan

Maintenant, nous connaissons des quantités importantes pour calculer la distance physique couverte par un ct-scan !

In [ ]:
scan_properties["r_distance"] = scan_properties.pixelspacing_r * scan_properties.rows
scan_properties["c_distance"] = scan_properties.pixelspacing_c * scan_properties["columns"]
scan_properties["area_cm2"] = 0.1* scan_properties["r_distance"] * 0.1*scan_properties["c_distance"]
scan_properties["slice_volume_cm3"] = 0.1*scan_properties.slice_thickness * scan_properties.area_cm2

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.distplot(scan_properties.area_cm2, ax=ax[0], color="purple")
sns.distplot(scan_properties.slice_volume_cm3, ax=ax[1], color="magenta")
ax[0].set_title("CT-slice area in $cm^{2}$")
ax[1].set_title("CT-slice volume in $cm^{3}$")
ax[0].set_xlabel("$cm^{2}$")
ax[1].set_xlabel("$cm^{3}$");

Nous avons des images avec des zones et des volumes de sliches extrêmement larges

## Surface et volume de la tranche du CT-scan - EDA <a class="anchor" id="scan_eda"></a>

### La plus petite et la plus grande zone de coupe CT

In [ ]:
max_path = scan_properties[
    scan_properties.area_cm2 == scan_properties.area_cm2.max()].patient_pth.values[0]
min_path = scan_properties[
    scan_properties.area_cm2 == scan_properties.area_cm2.min()].patient_pth.values[0]

min_scans = load_scans(min_path)
min_hu_scans = transform_to_hu(min_scans)

max_scans = load_scans(max_path)
max_hu_scans = transform_to_hu(max_scans)

background_water_hu_scans = max_hu_scans.copy()

In [ ]:
def set_manual_window(hu_image, custom_center, custom_width):
    w_image = hu_image.copy()
    min_value = custom_center - (custom_width/2)
    max_value = custom_center + (custom_width/2)
    w_image[w_image < min_value] = min_value
    w_image[w_image > max_value] = max_value
    return w_image

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,10))
ax[0].imshow(set_manual_window(min_hu_scans[np.int(len(min_hu_scans)/2)], -500, 1000), cmap="YlGnBu")
ax[1].imshow(set_manual_window(max_hu_scans[np.int(len(max_hu_scans)/2)], -500, 1000), cmap="YlGnBu");
ax[0].set_title("CT-scan with small slice area")
ax[1].set_title("CT-scan with large slice area");
for n in range(2):
    ax[n].axis("off")

### Perspectives

* En regardant une tranche de scan avec la plus petite et la plus grande surface, on peut voir que la grande tranche a beaucoup de région inutile couverte. Nous pourrions la recadrer.
* Bizarre... dans la deuxième image avec la grande surface, la région extérieure du tube du scanner n'est pas réglée sur la valeur de l'air mais plutôt sur une valeur située au milieu de la plage de -1000 à 1000.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.distplot(max_hu_scans[np.int(len(max_hu_scans)/2)].flatten(), kde=False, ax=ax[1])
ax[1].set_title("Large area image")
sns.distplot(min_hu_scans[np.int(len(min_hu_scans)/2)].flatten(), kde=False, ax=ax[0])
ax[0].set_title("Small area image")
ax[0].set_xlabel("HU values")
ax[1].set_xlabel("HU values");

* Pour l'exemple de l'OSIC, nous pouvons trouver : Ihh... il était réglé sur l'eau par défaut dans la grande image... pourquoi ? ! C'est mauvais ! Nous devons trouver une stratégie pour régler ce problème. Ce n'est pas bon que nous ayons parfois des régions extérieures qui ressemblent à de l'"eau" et parfois des régions qui ressemblent à de l'"air".

### Le plus petit et le plus grand volume de CT-slice

In [ ]:
max_path = scan_properties[
    scan_properties.slice_volume_cm3 == scan_properties.slice_volume_cm3.max()].patient_pth.values[0]
min_path = scan_properties[
    scan_properties.slice_volume_cm3 == scan_properties.slice_volume_cm3.min()].patient_pth.values[0]

min_scans = load_scans(min_path)
min_hu_scans = transform_to_hu(min_scans)

max_scans = load_scans(max_path)
max_hu_scans = transform_to_hu(max_scans)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,10))
ax[0].imshow(set_manual_window(min_hu_scans[np.int(len(min_hu_scans)/2)], -500, 1000), cmap="YlGnBu")
ax[1].imshow(set_manual_window(max_hu_scans[np.int(len(max_hu_scans)/2)], -500, 1000), cmap="YlGnBu");
ax[0].set_title("CT-scan with small slice volume")
ax[1].set_title("CT-scan with large slice volume");
for n in range(2):
    ax[n].axis("off")

je ne vois pas une grande différence. Peut-être que celle avec le grand volume de la tranche semble un peu plus floue. Mais comme ci-dessus, il y a une région du scanner extérieur qui a été réglée sur la valeur de l'eau (valeur HU de 0) au lieu de celle de l'air.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.distplot(max_hu_scans[np.int(len(max_hu_scans)/2)].flatten(), kde=False, ax=ax[1])
ax[1].set_title("Large slice volume")
sns.distplot(min_hu_scans[np.int(len(min_hu_scans)/2)].flatten(), kde=False, ax=ax[0])
ax[0].set_title("Small slice volume")
ax[0].set_xlabel("HU values")
ax[1].set_xlabel("HU values");

## Reconstruction 3D du CT-scans <a class="anchor" id="reconstruction"></a>

Le plot_3d fonctionne bien dans le Data Science Bowl 2017, mais dans notre cas, les résultats ne sont pas aussi bons. Cela dépend du seuil, mais jusqu'à présent, je ne sais pas pourquoi nos reconstructions ont souvent l'air floues ou montrent aussi des régions du tube

In [ ]:
def plot_3d(image, threshold=700, color="navy"):
    
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    
    verts, faces,_,_ = measure.marching_cubes_lewiner(p, threshold)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.2)
    mesh.set_facecolor(color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    plt.show()

In [ ]:
plot_3d(max_hu_scans)

Vous pouvez voir que celui-ci est très différent de celui-là

In [ ]:
old_distribution = max_hu_scans.flatten()

In [ ]:
example = train.dcm_path.values[0]
scans = load_scans(example)
hu_scans = transform_to_hu(scans)

In [ ]:
plot_3d(hu_scans)

Par rapport à la version précédente, celle-ci est bien plus belle. Traçons les distributions. Peut-être pouvons-nous comprendre ce qui ne va pas en les regardant :

In [ ]:
plt.figure(figsize=(20,5))
sns.distplot(old_distribution, label="weak 3d plot", kde=False)
sns.distplot(hu_scans.flatten(), label="strong 3d plot", kde=False)
plt.title("HU value distribution")
plt.legend();

In [ ]:
print(len(max_hu_scans), len(hu_scans))

Je pense que nous devons comprendre l'algorithme de marching_cubes_lewiner pour comprendre pourquoi l'intrigue fonctionne parfois bien et parfois pas. Mais je pense que ce n'est pas vraiment important pour la compétition elle-même. Pour l'instant, je n'aime pas passer plus de temps sur ce sujet. Il est peut-être plus important de garder à l'esprit que les distributions globales peuvent être différentes.

## Rééchantillonnage de la taille des voxels

In [ ]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness] + list(scan[0].PixelSpacing), dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

In [ ]:
img_resampled, spacing = resample(max_hu_scans, scans, [1,1,1])
print("Shape before resampling\t", hu_scans.shape)
print("Shape after resampling\t", img_resampled.shape)

## Tissue segmentation <a class="anchor" id="segmentation"></a>

In [ ]:
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None
    
def fill_lungs(binary_image):
    image = binary_image.copy()
    # For every slice we determine the largest solid structure
    for i, axial_slice in enumerate(image):
        axial_slice = axial_slice - 1
        labeling = measure.label(axial_slice)
        l_max = largest_label_volume(labeling, bg=0)

        if l_max is not None: #This slice contains some lung
            image[i][labeling != l_max] = 1
    return image


In [ ]:
def segment_lung_mask(image):
    segmented = np.zeros(image.shape)   
    
    for n in range(image.shape[0]):
        binary_image = np.array(image[n] > -320, dtype=np.int8)+1
        labels = measure.label(binary_image)
        
        background_label_1 = labels[0,0]
        background_label_2 = labels[0,-1]
        background_label_3 = labels[-1,0]
        background_label_4 = labels[-1,-1]
    
        #Fill the air around the person
        binary_image[background_label_1 == labels] = 2
        binary_image[background_label_2 == labels] = 2
        binary_image[background_label_3 == labels] = 2
        binary_image[background_label_4 == labels] = 2
    
        #We have a lot of remaining small signals outside of the lungs that need to be removed. 
        #In our competition closing is superior to fill_lungs 
        selem = disk(4)
        binary_image = closing(binary_image, selem)
    
        binary_image -= 1 #Make the image actual binary
        binary_image = 1-binary_image # Invert it, lungs are now 1
        
        segmented[n] = binary_image.copy() * image[n]
    
    return segmented

### Comprendre la segmentation étape par étape:

In [ ]:
plt.figure(figsize=(20,5))
sns.distplot(hu_scans[20], kde=False)
plt.title("Example HU value distribution");
plt.xlabel("HU-value")
plt.ylabel("count")

Avec -320, nous séparons les poumons (-700) / l'air (-1000) et les tissus avec des valeurs proches de l'eau (0).

In [ ]:
binary_image = np.array((hu_scans[20]>-320), dtype=np.int8) + 1
np.unique(binary_image)

Les poumons ont des valeurs de 1 ainsi que les milieux atmosphériques. En revanche, les milieux par défaut semblables à ceux de l'eau et de nombreux autres tissus ou fluides organiques ont des valeurs de 2. Comme nous aimons seulement segmenter les poumons, nous devons éliminer le milieu. Dans le cas de l'air et des valeurs par défaut similaires à l'air, nous devons définir manuellement leurs valeurs à 2. Nous pouvons le faire en étiquetant les régions connectées dans l'image binaire et en extrayant les étiquettes de chaque région qui correspond aux coins de la tranche d'image 2D 

In [ ]:
labels = measure.label(binary_image)

background_label_1 = labels[0,0]
background_label_2 = labels[0,-1]
background_label_3 = labels[-1,0]
background_label_4 = labels[-1,-1]

Nous pouvons maintenant définir toutes les régions étiquetées de l'image binaire qui correspondent à ces étiquettes d'angle à la valeur "non poumon" 2 

In [ ]:
binary_image_2 = binary_image.copy()
binary_image_2[background_label_1 == labels] = 2
binary_image_2[background_label_2 == labels] = 2
binary_image_2[background_label_3 == labels] = 2
binary_image_2[background_label_4 == labels] = 2

Le résultat de ces étapes se présente comme suit 

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,7))
ax[0].imshow(binary_image, cmap="binary", interpolation='nearest')
ax[1].imshow(labels, cmap="jet", interpolation='nearest')
ax[2].imshow(binary_image_2, cmap="binary", interpolation='nearest')

ax[0].set_title("Binary image")
ax[1].set_title("Labelled image");
ax[2].set_title("Binary image - background removed");

### Perspectives

1. La première image montre le binaire brut. Dans ce cas, nous trouvons l'air comme arrière-plan et nous devons le régler sur la valeur "non poumon" de 2.
2. Pour cela, nous étiquetons toutes les régions connectées dans l'image binaire. Il existe de nombreuses régions étiquetées, mais les seules qui nous intéressent sont les 4 régions d'angle à (0,0), (0,500), (500,0) et (500,500).
3. La connaissance des étiquettes correspondantes nous aide à régler manuellement le fond à la valeur 2 (noir).
4. Au final, nous pouvons voir que les poumons sont blancs (1), mais nous trouvons encore beaucoup de signaux restants qui correspondent à des tissus corporels qu'il nous faut encore enlever.

Pour supprimer les signaux, nous pouvons utiliser la fermeture morphologique. Si vous aimez jouer avec la valeur du disque. Elle doit être suffisamment grande pour annuler les signaux du corps mais suffisamment petite pour garder suffisamment de détails à l'intérieur des poumons

In [ ]:
selem = disk(4)
closed_binary_2 = closing(binary_image_2, selem)

closed_binary_2 -= 1 #Make the image actual binary
closed_binary_2 = 1-closed_binary_2 # Invert it, lungs are now 1

Comparons avec la méthode de remplissage des poumons de Guidos:

In [ ]:
filled_lungs_binary = fill_lungs(binary_image_2)

Et avec la suppression de sa poche d'air 

In [ ]:
air_pocket_binary = closed_binary_2.copy()
# Remove other air pockets insided body
labels_2 = measure.label(air_pocket_binary, background=0)
l_max = largest_label_volume(labels_2, bg=0)
if l_max is not None: # There are air pockets
    air_pocket_binary[labels_2 != l_max] = 0

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,7))

ax[0].imshow(closed_binary_2, cmap="binary", interpolation='nearest')
ax[1].imshow(filled_lungs_binary, cmap="binary", interpolation='nearest')
ax[2].imshow(air_pocket_binary, cmap="binary", interpolation='nearest')


ax[0].set_title("Morphological closing");
ax[1].set_title("Guidos filling lung structures");
ax[2].set_title("Guidos air pocket removal");

### Perspectives

* La fermeture morphologique a mieux fonctionné que la méthode Guidos. Mais il nous manque souvent beaucoup d'informations à l'intérieur des poumons, juste pour éliminer ces signaux corporels. Il y a certainement place pour des améliorations ! ;-)
* En revanche, la méthode du fill-lung a des problèmes avec les signaux restants du corps et ne donne pas ce que nous aimons obtenir.
* En outre, l'élimination des poches d'air ne fonctionne pas bien non plus.



### solution Finale

Et voici à quoi cela ressemble si nous masquons l'image originale avec les poumons binaires segmentés dans notre exemple en 2D

In [ ]:
segmented = segment_lung_mask(np.array([hu_scans[20]]))

fig, ax = plt.subplots(1,2,figsize=(20,10))
ax[0].imshow(hu_scans[20], cmap="Blues_r")
ax[1].imshow(segmented[0], cmap="Blues_r");

Et nous pouvons également vérifier l'aspect de l'affaire 3D

In [ ]:
segmented_lungs = segment_lung_mask(hu_scans)

In [ ]:
fig, ax = plt.subplots(6,5, figsize=(20,20))
for n in range(6):
    for m in range(5):
        ax[n,m].imshow(segmented_lungs[n*5+m], cmap="Blues_r")

Il est intéressant de noter que nous avons encore quelques signaux en dehors des poumons

In [ ]:
plot_3d(segmented_lungs, threshold=-600)

améliorer la segmentation

# Génération d'un ensemble de données pour les fichiers prétraités <a class="anchor" id="datagenerator"></a>


## Traitement des différentes tailles d'images <a class="anchor" id="image_sizes"></a>

Pour générer les données, nous devons réexaminer les différentes tailles d'images : Pour l'OSIC, nous avons deux grands groupes de taille et quelques petites valeurs aberrantes. Par exemple, nous pourrions redimensionner ou recadrer manuellement les valeurs aberrantes et trouver une stratégie pour les deux grands groupes. Examinons à nouveau les tailles :

In [ ]:
image_sizes = scan_properties.groupby(["rows", "columns"]).size().sort_values(ascending=False)
image_sizes

Pour l'OSIC, il est intéressant de constater que nous avons deux types de modèles différents. Dans la compétition RSNA, toutes les images d'entraînement sont de forme 512, 512. Vous n'avez donc pas besoin d'explorer davantage les tailles des images. Mais peut-être est-il encore utile de parcourir les images pour trouver des idées de bonnes augmentations.

In [ ]:
plt.figure(figsize=(8,8))
for n in counts.index.values:
    for m in counts.columns.values:
        plt.scatter(n, m, s=counts.loc[n,m], c="dodgerblue", alpha=0.7)
plt.xlabel("rows")
plt.ylabel("columns")
plt.title("Pixel area of ct-scan per patient");
plt.plot(np.arange(0,1400), '-.', c="purple", label="squared")
plt.plot(888 * np.ones(1400), '-.', c="crimson", label="888 rows");
plt.legend();

Ce genre de modèles évidents vaut toujours la peine d'être examiné. Peut-être pouvons-nous trouver une sorte de règle qui nous permette de créer une bonne stratégie de redimensionnement. Voici un petit observateur qui vous permet de parcourir les fichiers en exécutant à nouveau la cellule de code :

In [ ]:
class ImageObserver:
    
    def __init__(self, scan_properties, batch_size):
        self.scan_properties = scan_properties
        self.batch_size = batch_size
    
    def select_group(self, group=(512,512)):
        self.group = group
        self.name = "rows {}, columns {}".format(group[0], group[1])
        self.batch_shape = (self.batch_size, group[0], group[1])
        self.selection = self.scan_properties[
            (self.scan_properties["rows"]==group[0]) & (self.scan_properties["columns"]==group[1])
        ].copy()
        self.patient_pths = self.selection.patient_pth.unique()
    
    
    def get_loader(self):
        
        idx=0
        images = np.zeros(self.batch_shape)
        
        for path in self.patient_pths:
            
            scans = load_scans(path)
            hu_scans = transform_to_hu(scans)
            images[idx,:,:] = hu_scans[0]
            
            idx += 1
            if idx == self.batch_shape[0]:
                yield images
                images = np.zeros(self.batch_shape)
                idx = 0
        if idx > 0:
            yield images

In [ ]:
my_choice = image_sizes.index.values[0]
print(my_choice)
to_display = 4

In [ ]:
observer = ImageObserver(scan_properties, to_display)
observer.select_group(my_choice)
observer_iterator = observer.get_loader()

Il suffit de relancer la cellule suivante, pour observer le prochain lot d'images :

In [ ]:
images = next(observer_iterator)

In [ ]:
fig, ax = plt.subplots(1,to_display,figsize=(20,5))


for m in range(to_display):
    image = images[m]
    ax[m].imshow(set_manual_window(image, -500, 1000), cmap="YlGnBu")
    ax[m].set_title(observer.name)

### Insights

* Les grandes tailles d'images carrées présentent souvent des résolutions plus élevées qu'avec 512 lignes et 512 colonnes. Le redimensionnement en fonction des deux grands groupes (512, 512) ou (768, 768) est ici logique.
* Dans les cas non carrés, les cultures centrales devraient fonctionner au mieux car elles n'ont que des valeurs de fond plus importantes mais appartiennent toujours aux grands groupes de la région du scanner interne.

### pour en savoir plus sur les unités Hounsfield

In [ ]:
from IPython.display import HTML
HTML('<center><iframe width="700" height="400" src="https://www.youtube.com/embed/KZld-5W99cI?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe></center>')